# Q Peak Plotting Example (Empkins dataset)

## Setup and imports

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from fau_colors import cmaps

from pepbench.datasets import EmpkinsDataset
from pepbench.algorithms.ecg import QPeakExtractionVanLien2013
from pepbench.plotting.algorithms import plot_q_peak_extraction_vanlien2013

%matplotlib widget
%load_ext autoreload
%autoreload 2

## Plotting style

In [ ]:
plt.close("all")
palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)
plt.rcParams["figure.figsize"] = (10, 5)

## Point the dataset class to the local Empkins dataset folder inside example_data/
This cell auto-detects the repository root so the notebook works even if Jupyter's CWD isn't the repo root.

In [ ]:
empkins_base_path = "/Users/joana/Desktop/University/Master/Forschungspraktikum/Data/Empkins_Dataset"
# TODO: update this path to where your Empkins dataset is stored

## Instantiate EmpkinsDataset
use_cache=False for reproducible load in this example

In [ ]:
dataset = EmpkinsDataset(empkins_base_path, use_cache=False, only_labeled=True)
dataset

### Show available participant/condition/phase rows and pick the first one

In [ ]:
print("Dataset index (first 10 rows):")
display(dataset.index.head(10))

## Select the first index entry
(participant, condition, phase)

In [ ]:
row = dataset.index.iloc[0]
participant, condition, phase = row["participant"], row["condition"], row["phase"]
print("Using:", participant, condition, phase)

## Create a subset for that single participant/condition/phase

In [ ]:
datapoint = dataset.get_subset(participant=participant, condition=condition, phase=phase)

## Run Q-peak extraction (Van Lien 2013)

In [ ]:
q_algo = QPeakExtractionVanLien2013()
# use the computed heartbeat segmentation
q_algo.extract(ecg=datapoint.ecg, heartbeats=datapoint.heartbeats, sampling_rate_hz=datapoint.sampling_rate_ecg)

## Inspect detected points (first rows)

In [ ]:
display(q_algo.points_.head())

## Plot a few heartbeats to visualise detected Q-peaks
determine the first three heartbeat_ids available in the reference_heartbeats index

In [ ]:
hb_ids = list(datapoint.reference_heartbeats.index.get_level_values("heartbeat_id").unique())[:3]
print("Plotting heartbeat ids:", hb_ids)
fig, ax = plot_q_peak_extraction_vanlien2013(datapoint, heartbeat_subset=hb_ids, normalize_time=True)
plt.show()

## Lower-level plotting helpers
Demonstrate the primitive plotting helpers that are used by the high-level plotting wrappers. This shows how to add R/Q peaks and ICG C/B points and heartbeat borders manually on axes.

In [ ]:
from pepbench.plotting._utils import (
    _add_ecg_q_peaks,
    _add_ecg_r_peaks,
    _add_heartbeat_borders,
    _add_icg_b_points,
    _add_icg_c_points,
    _get_data,
    _get_reference_labels,
    _get_heartbeats,
    _get_heartbeat_borders,
)
from pepbench.algorithms.icg import CPointExtractionScipyFindPeaks

# prepare a small heartbeat subset (reuse hb_ids from above)
# hb_ids is expected to be defined in the notebook earlier; if not, compute it again
try:
    hb_ids
except NameError:
    hb_ids = list(datapoint.reference_heartbeats.index.get_level_values("heartbeat_id").unique())[:3]

# get sliced ECG/ICG data (no time normalization here)
ecg_data, icg_data = _get_data(datapoint, normalize_time=False, heartbeat_subset=hb_ids)
# relative heartbeats for the subset
heartbeats_rel = _get_heartbeats(datapoint, heartbeat_subset=hb_ids, normalize=True)
# heartbeat border x positions
heartbeat_borders = _get_heartbeat_borders(ecg_data, heartbeats_rel)

# reference labels for Q/B points adjusted to the subset
refs = _get_reference_labels(datapoint, heartbeat_subset=hb_ids)
q_ref = refs["q_peaks"]
b_ref = refs["b_points"]

# compute C-points for the ICG subset using the scipy find-peaks C-point extractor
c_algo = CPointExtractionScipyFindPeaks()
# extract on the sliced icg and the relative heartbeats
c_algo.extract(icg=icg_data, heartbeats=heartbeats_rel, sampling_rate_hz=datapoint.sampling_rate_icg)
c_points = c_algo.points_["c_point_sample"].dropna().astype(int)

# Now plot ECG (top) and ICG (bottom) and overlay markers using the low-level helpers
fig, axs = plt.subplots(nrows=2, sharex=True, figsize=(10, 6))

# ECG top
axs[0].plot(ecg_data.index, ecg_data.squeeze(), color=cmaps.tech[1], label="ECG")
_add_ecg_r_peaks(ecg_data, heartbeats_rel["r_peak_sample"], ax=axs[0], r_peak_marker="X", r_peak_color=cmaps.tech[3])
_add_ecg_q_peaks(ecg_data, q_ref, ax=axs[0], q_peak_marker="o", q_peak_color=cmaps.tech[4])
_add_heartbeat_borders(heartbeat_borders, ax=axs[0], heartbeat_border_color=cmaps.tech[2])
axs[0].set_ylabel("ECG amplitude")
axs[0].legend()

# ICG bottom
axs[1].plot(icg_data.index, icg_data.squeeze(), color=cmaps.tech[0], label="ICG")
_add_icg_c_points(icg_data, c_points, ax=axs[1], c_point_marker="X", c_point_color=cmaps.tech[3])
_add_icg_b_points(icg_data, b_ref, ax=axs[1], b_point_marker="o", b_point_color=cmaps.tech[4])
_add_heartbeat_borders(heartbeat_borders, ax=axs[1], heartbeat_border_color=cmaps.tech[2])
axs[1].set_ylabel("ICG amplitude")
axs[1].legend()

plt.tight_layout()
plt.show()

## Normalize time
Show how time-normalization affects plotted signals and helper indices: raw indices vs seconds-from-zero.

In [ ]:
# get non-normalized and normalized slices for the same heartbeat subset
ecg_raw, icg_raw = _get_data(datapoint, normalize_time=False, heartbeat_subset=hb_ids)
ecg_norm, icg_norm = _get_data(datapoint, normalize_time=True, heartbeat_subset=hb_ids)

print("Raw ECG index sample (first 5):", list(ecg_raw.index[:5]))
print("Normalized ECG index sample (first 5):", list(ecg_norm.index[:5]))

# quick visual comparison
fig, axs = plt.subplots(1, 2, figsize=(12, 3), sharey=True)
axs[0].plot(ecg_raw.index, ecg_raw.squeeze(), color=cmaps.fau[0])
axs[0].set_title("Raw time (samples)")
axs[0].set_xlabel("Sample index")

axs[1].plot(ecg_norm.index, ecg_norm.squeeze(), color=cmaps.fau[0])
axs[1].set_title("Normalized time (seconds from 0)")
axs[1].set_xlabel("Time (s)")

plt.tight_layout()
plt.show()

### Notes:
- You can swap `QPeakExtractionVanLien2013` for other algorithms available in `pepbench.algorithms.ecg` (e.g., `QPeakExtractionMartinez2004Neurokit`, `QPeakExtractionForouzanfar2018`).
- To use non-reference heartbeats, use `datapoint.heartbeats` instead of `datapoint.reference_heartbeats`.
- If the dataset is large, set `use_cache=True` when instantiating `EmpkinsDataset` for faster repeated loads.
